# Training Variational Autoencoders Using TensorFlow2.X and Keras

- This notebook focuses on training a variational autoencoder.
- The Fashion MNIST dataset will be utilized due to its lightweight nature.
- This assists learners to learn key concepts in VAE training without lengthy training processes or GPU dependencies.

## Step 1: Import Required Libraries
- Import necessary libraries and modules for numerical computations, operating system interactions, data visualization, and deep learning.
- The OS module is imported to interact with the operating system.
- The **%matplotlib inline** is a Jupyter Notebook magic command that enables inline plotting.
- TensorFlow is imported as the main deep learning framework.
- Keras is imported from TensorFlow to utilize the Keras API for building and training models.
- An assertion is made to ensure that the TensorFlow version is 2.0 or higher.


In [15]:
import tensorflow as tf
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow import keras
assert tf.__version__>="2.0"
K = keras.backend # backend

## Step 2: Load the Fashion MNIST Dataset and Split It into Training and Testing Sets
- Fashion MNIST Dataset is available in Keras and can be used from the Keras library instead of explicitly downloading it from the source. After downloading the dataset, normalize it by diving 255 and split the dataset
- Convert the pixel values to floating-point numbers between 0 and 1
- Split the training set into training and validation sets

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [3]:
X_train.shape

(55000, 28, 28)

## Step 3: Visualize Original and Reconstructed Images Using a Model
- Plot the image using a binary color map
- Generate reconstructions using the model for a specified number of images
- Create a figure with subplots to display the original images and their reconstructions
- Plot the original image in the first row of subplots
- Plot the reconstructed image in the second row of subplots


In [4]:
def plot_image(image):
    plt.imshow(image, cmap="binary")
    plt.axis("off")

def show_output(model, images=X_valid, n_images=10):
    reconstructions = model.predict(images[:n_images])
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(images[image_index])
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index])

## Step 4: Implement Gaussian Sampling Layer for a Variational Autoencoder
- This function is used for sampling the code from a standard normal distribution with a given mean and standard deviation.

In [5]:
class gaussian_sample(keras.layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        return K.random_normal(tf.shape(log_var)) * K.exp(log_var / 2) + mean

## Step 5: Set the Random Seed for Both Tensorflow and Numpy to 50
- Set the seed to make sure that results are consistent and replicable
- Input embedding size


In [6]:

tf.random.set_seed(50)
np.random.seed(50)

latent_size = 6


## Step 6: Define the Encoder of VAE
- Define the input and hidden layers for the encoder with the number of nodes in each layer as well as the activation function to be used.
- After calculating the mean and standard deviation, we use **gaussian_sample** layer to produce the sampled coding.
- Create an encoder that outputs sampled embeddings.



In [ ]:
inputs = keras.layers.Input(shape=[28, 28])
z = keras.layers.Flatten()(inputs)
z = keras.layers.Dense(250, activation="relu")(z)
z = keras.layers.Dense(150, activation="relu")(z)

latent_mean = keras.layers.Dense(latent_size)(z)
latent_log_var = keras.layers.Dense(latent_size)(z)


latent = gaussian_sample()([latent_mean, latent_log_var])
vae_encoder = keras.models.Model(
    inputs=[inputs], outputs=[latent_mean, latent_log_var, latent])

## Step 7: Define the Decoder
- Define the input, hidden, and output layers of the decoder
- Output of the decoder must be reshaped to the same dimensions as the encoder's input

In [8]:
decoder_inputs = keras.layers.Input(shape=[latent_size])
x = keras.layers.Dense(150, activation="relu")(decoder_inputs)
x = keras.layers.Dense(250, activation="relu")(x)
x = keras.layers.Dense(28*28, activation="sigmoid")(x)
outputs = keras.layers.Reshape([28, 28])(x)
vae_decoder = keras.models.Model(inputs=[decoder_inputs], outputs=[outputs])


## Step 8: Build the Autoencoder
- Use the encoder to generate the sampled embeddings
- Provide sampled embeddings as the only input to the decoder
- Create the final VAE model

In [9]:
e_mean, e_log_var, latent = vae_encoder(inputs)
reconstructions = vae_decoder(latent)
variational_ae = keras.models.Model(inputs=[inputs], outputs=[reconstructions])


## Step 9: Define the Loss, Compile the Model, and Kick-off the Training
- Generate sampled embeddings using the encoder
- Encode the input data to obtain the mean and variance of the latent space
- Define a function for sample embeddings from the latent space
- The sampled_embeddings variable now contains the generated embeddings


In [16]:
latent_loss = -0.5 * K.sum(
1 + latent_log_var - K.exp(latent_log_var) - K.square(latent_mean),
    axis=-1)
variational_ae.add_loss(K.mean(latent_loss) / 784.)
variational_ae.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
history = variational_ae.fit(X_train, X_train, epochs=50, batch_size=128,
                             validation_data=(X_valid, X_valid))

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.ops`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


##  Step 10: Display the Output
- This function likely generates reconstructions of the input images using the model and plots them.

In [ ]:
show_output(variational_ae)
plt.show()

## Step 11: Generate a Few Images Using Trained VAE
- Calculate the number of columns for subplots
- Calculate the number of rows for subplots
- Remove the last dimension if it's one
- Create a figure with the specified size
- Create a subplot for each image
- Plot the image using a binary color map
- Turn off the axis

In [ ]:
def plot_images(images, n_cols=None):
    n_cols = n_cols or len(images)
    n_rows = (len(images) - 1) // n_cols + 1
    if images.shape[-1] == 1:
        images = np.squeeze(images, axis=-1)
    plt.figure(figsize=(n_cols, n_rows))
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(image, cmap="binary")
        plt.axis("off")

## Step 12: Display the Generated Images in a Grid with Four Columns
- Set the TensorFlow random seed to 50
- Generate random embeddings
- Decode embeddings to generate images
- Plot the generated images in a grid of four columns


In [ ]:
#tf.random.set_seed(50)

new_inputs = tf.random.normal(shape=[1, latent_size],mean=0.0,stddev=1) # Normal distribution with mu=0 and sigma=2
images = vae_decoder(new_inputs).numpy() # Use the defined, trained decoder only
plot_images(images, 4)

Congratulations! You've trained and tested a variational autoencoder.